# Imaging Lab 3: Multipixel Scanning

## EECS 16A: Designing Information Devices and Systems I, Fall 2022

<!--- Raghav Gupta raghavgupta@berkeley.edu --->


## Table of Contents

* [Instructions](#instructions)
* [Lab Policies](#policies)
* [Overview](#overview)
* [Task 1: Generating Multipixel Scanning Matrices](#matrixGenIntro)
    * [Task 1a: Imaging Mask Matrix Practice](#simpleMatrixGen)
    * [Task 1b: Generating a Random Binary Mask Matrix](#randomBinaryMatrixGen)
* [Task 2: Imaging Simulator](#simulatorIntro)
    * [Task 2a: Constructing an Ideal Sensor Model](#idealSensor)
        * [Image Reconstruction Using the Ideal Sensor Model + Matrix Inverse](#idealReconstruction)
    * [Task 2b: Handling System Non-Idealities](#nonidealities)
        * [Noise *(Why So Grainy? ☹)*](#noiseSimulation)
    * [Task 2c: Eigenanalysis & the Robustness of Inverse-Based Reconstruction](#eigenanalysis)
        * [Graphical Interpretation](#graphicalInterpretation)
        * [Revisiting the Identity Matrix](#revisitingIdentity)
        * [Comparing Scanning Matrices](#comparingScanning)
* [Task 3: Scanning Images](#scanningImages)
    * [Task 3a: Single Pixel Sanity Check](#singlePixel)
    * [Task 3b: Real Multipixel Imaging](#realImaging)
* [Task 4: Understanding Multipixel Use-Cases](#useCases)
* [Checkoff](#checkoff)

<a id='instructions'></a>
## Instructions

* **Gently disinfect all commonly touched surfaces** at your lab station using the provided cleaning wipes.


* Complete this lab by filling in all of the required sections, marked with `"YOUR CODE HERE"` or `"YOUR ANSWER HERE"`.


* When you finish, submit a checkoff request to get checked off for this lab. Be ready to answer a few questions to show your understanding of each section.


* Labs will be graded based on completion for teams of 2 students.

<a id='policies'></a>
## Lab Policies
* **YOU MUST ATTEND THE LAB SECTION YOU ARE ENROLLED IN. If you anticipate missing a section, please notify your GSI in advance.**
* **You are required to return all parts checked out at the beginning of the lab section unless told otherwise.**
* **You are free to stay for the full allotted time and hack around with the lab equipment, but please reserve the GSI's time for lab-related questions.**
* **Food and drinks are not allowed in the lab.** 
* **Clean up, turn off all equipment, and log off of computers before leaving.**

<a id='overview'></a>
# <span style='color:blue'>Overview</span>

Recall that in the last lab, you illuminated the object pixel-by-pixel. This week, you'll flex your linear algebra skills and try something different. You will experiment with imaging methods that illuminate *multiple pixels* at a time. You'll find that if we design our masks in a clever way, our imaging system can be much more robust to noise than the single-pixel approach. You will generate a binary mask matrix that the projector will use to illuminate multi-pixel patterns onto your object. Before scanning your custom images, you'll walk through a basic multi-pixel imaging simulation to understand how it works, delve deeper into the differences between ideal and non-ideal imaging, and understand why certain matrices are better than others at imaging in noisy systems.

*Note: A lot of the code to complete this lab will be provided for you to run. However, looking over the code to try to understand what it does is **highly encouraged**. Additionally, we will be writing **functions** to enable multiple parts of this lab to reuse the same code with minimal copy + pasting.*

**<span style = "color: red">Run the following code block to get access to several pre-written functions and helper libraries.</span>**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%run scripts/helpers.py
%matplotlib inline

# <a id = 'matrixGenIntro'><span style = "color: blue">Task 1: Generating Multipixel Scanning Matrices</span></a>

**Note: This lab will use 0-indexing, as Python uses 0-indexing.**

Recall that we can define our imaging system by the following mathematical model:

$$ H \vec{i} = \vec{s} $$ 

$H$ is the imaging mask matrix, $\vec{i}$ is our image in column vector form, and $\vec{s}$ is the sensor output also in column vector form.

In Imaging 2, we scanned our image by highlighting one pixel at a time in a mask. Each row $H_k$ of $H$ defined a 1-D representation of each mask. This meant that scanning an image with $n$ pixels would require an $H$ with $n$ rows. Since we project one mask at a time onto our image, we would need to do exactly $n$ scans. Take a $2\times2$ image for example: $H$ would need exactly 4 rows, and we would make 4 scans.

Let's try something different. We'll still do $n$ scans of our image, but let's try to highlight more than one pixel per mask. We still want to be able to reconstruct our image from our sensor values. So the question is: how do you choose which pixels to illuminate with each mask?

Begin by assigning each pixel value in the 2x2 image to a variable, $p_{ij}$, where $i$ is the row and $j$ is the column associated with the pixel location. <br/><br/>

<center>
    <b>2x2 Image</b>
<img src="images/img_4x4_new.png" align="center" style="height:200px" />
</center>

<!--
In matrix form, the 2x2 image will look like this:
$$\begin{bmatrix} p_{00} & p_{01} \\ p_{10} & p_{11} \end{bmatrix}$$
-->

In our mathematical model above, we represent the 2x2 image as the 1D column vector: 

$$\vec{i} = \begin{bmatrix} p_{00} \\ p_{01} \\ p_{10} \\ p_{11} \end{bmatrix}$$

Likewise, the sensor reading column vector is represented as:

$$\vec{s} = \begin{bmatrix} s_0 \\ s_1 \\ s_2 \\ s_3 \end{bmatrix}$$

Where the sensor reading from the $k$th mask is $s_k$. In the example above, the sensor reading from the 2nd mask is $s_2$. (We consider $s_0$ to be the 0th mask.)

To illustrate the relationship between the mask matrix $H$ (with per-row imaging masks $H_k$), the image vector $\vec{i}$, and the sensor reading vector $\vec{s}$, we provide you with this **Example System of Linear Equations:**

$$
\begin{align} 
s_0 & = p_{00}\\
s_1 & = p_{00} + p_{01}\\
s_2 & = p_{00} + p_{10}\\
s_3 & = p_{01} + p_{10} + p_{11}
\end{align}
$$

**<span style = "color: red">*IMPORTANT*: The above system of equations is only an example! It only serves an illustrative purpose for this section of the lab. Please do not use it for the rest of the lab.</span>**

How would you represent the above as a mask matrix $H$? Convince yourself that the image below does just that (where a **white pixel** represents a value of **1** and a **black pixel** represents a value of **0**).</font><br/><br/>

<center>
    <b>Imaging Mask Matrix $H$ for the Example System of Linear Equations</b>
<img src="images/mask_sample_4x4.png" align="center" style="height:200px" />
</center>

Recall that each row of our mask matrix represents a mask in 1-D form. We must *reshape* each row $H_k$ of $H$ into the 2-D mask (Mask $k$) itself before projecting it onto the image. To make sense of the $H$ matrix, it is helpful to look at each mask individually. Let's consider $H_0$, the 0th row of $H$. When we reshape the 1-D $1\times4$ row into a $2\times2$ mask, we get Mask 0 (below on the left). <br/><br/>

<center>
    <b>Individual Masks for the Example System of Linear Equations</b>
<img src="images/H_4x4_split.png" align="center" style="height:200px" />
</center>

Now we can see that 

$$H_k \vec{i} = s_k$$ 

represents one of the equations in our system. 

For example, the equation for $s_0$ only depends on one pixel, $p_{00}$, i.e. the top-left pixel of our $2\times2$ image. We can represent it algebraically as:
    
$$s_0 = H_0 \vec{i}$$

where $H_0 = \begin{bmatrix} 1 & 0 & 0 & 0 \end{bmatrix}$.

So numerically, we can represent the equation as:

$$s_0 = \begin{bmatrix} 1 & 0 & 0 & 0 \end{bmatrix} \begin{bmatrix} p_{00} \\ p_{01} \\ p_{10} \\ p_{11} \end{bmatrix}$$

## <a id = 'simpleMatrixGen'><span style = "color: blue">Task 1a: Imaging Mask Matrix Practice</span></a>

Now that we have an understanding of how to approach multipixel scanning, let's test our approach on a new system of equations:

<center>
<b>Lab 3 System of Equations</b>
$$
\begin{align}
s_0 & = p_{00} + p_{01} + p_{10}\\
s_1 & = p_{00} + p_{11}\\
s_2 & = p_{01} + p_{11}\\
s_3 & = p_{10} + p_{11}
\end{align}
$$

**<span style="color: red">For a 2x2 image represented by $\vec{i}$, create the matrix `H` such that $ H \vec{i} = \vec{s} $ represents the `Lab 3 System of Equations` above.</span>**
    
*Hint: Look up how to use the `np.array` function.*

In [ ]:
# TODO: Create H (4x4) for the Lab 3 System of Equations --------------------
H_new = # YOUR CODE HERE

# Show H
plt.imshow(H_new, cmap = "gray", interpolation = "nearest")
plt.title("4x4 H")

# Run autograder
test_H_new(H_new)

As stated above, we will *reshape* rows $H_k$ of the mask matrix, $H$, into the individual masks themselves. 

**<span style="color: red">You will help write a function `showMasks` that enables you to iterate through the 4 individual masks and display them as 2x2 images (1 TODO). Double check that the generated masks make sense visually and have the expected number of illuminated pixels. The `showMasks` function will be reused later.</span>**

*Hint: Reference your code from the part of Imaging 2 where you checked to make sure that the scanning matrix was producing the correct pattern by displaying each of the individual masks. You might want to check out the command `np.reshape`.* 

In [ ]:
# Inputs
#  `H`: Mask matrix
#  `X`: Image width
#  `Y`: Image height
#  `numMasksShown`: Number of individual masks to display (starting from `H` row 0)
def showMasks(H, X, Y, numMasksShown):
    plt.figure(figsize = (18, 12)) 
    # Use this for loop to iterate through the first `numMasksShown` rows of `H` 
    # you want to display.
    for k in range(numMasksShown):
        plt.subplot(numMasksShown, numMasksShown, k + 1)
    
        # TODO: Reshape the `k`th row of `H` to be shown in 2D --------------------
        # Hint: `X` is the width of the image you want to observe; `Y` is its height
        mask = # YOUR CODE HERE
  
        plt.imshow(mask, cmap = "gray", interpolation = "nearest")
        # Title also prints number of illuminated (white) pixels per mask
        plt.title("Mask " + str(k) + ": " + str(np.sum(H[k])) + " Illuminated Pixels")
    plt.show()

# Show individual masks    
showMasks(H = H_new, X = 2, Y = 2, numMasksShown = 4)

## <a id = 'randomBinaryMatrixGen'><span style = "color: blue">Task 1b: Generating a Random Binary Mask Matrix</span></a>

A 2x2 image is not very interesting to scan, so we will instead try to scan a 32x32 region. Note that this image has different dimensions compared to last week's image!

**<span style="color: red">To scan a 32x32 image, what dimensions must our scanning matrix $H$ have? What does the number of rows of $H$ correspond to? What does the number of columns correspond to? What do the elements in each column of $H$ represent?</span>**

`YOUR ANSWER HERE`

Since we'd like to use a sufficiently interesting set of masks and you *really* don't want to be constructing such a large matrix by hand, we will provide you with a function that generates a random binary mask matrix $H$ for you, given dimensions `(X, Y)` corresponding to your image's width/height, and, as we'll go into later, a parameter for the average number of illuminated pixels per scan. The resulting matrix $H$ will consist entirely of 0's and 1's, where 1's are randomly interspersed among 0's, and each row will contain approximately **`avg1sPerRow`** (see function arguments) # of 1's. Not all rows will contain the same number of 1's!

**<span style="color: red">Run the `generateRandomBinaryMask` function and visually inspect that the generated `randomH` (with approximately 300 pixels illuminated per scan) has the right dimensions & visually looks random. Don't worry too much about how this function is actually implemented, but you can check out the code in `scripts/helpers.py`.</span>** 

In [ ]:
# Generate the randomH mask
randomH = generateRandomBinaryMask(avg1sPerRow = 300)

**<span style="color: red">Use the `showMasks` function created earlier to show the first 4 individual masks (rows 0 to 3 of `randomH`) as 32x32 images.</span>**

In [ ]:
# TODO: Reuse the `showMasks` function from earlier to display the first 4 masks of randomH. -------

# YOUR CODE HERE


Think back to the representation of the imaging system as taking a matrix-vector product. Recall that in the Imaging 2 lab, you reconstructed the image column vector $\vec{i}$ from the sensor reading vector $\vec{s}$ by applying the equation:

$$\vec{i} = H^{-1} \vec{s}$$

You used the **identity** matrix for $H$, for which the inverse $H^{-1}$ exists. In order to apply the same reconstruction method assuming a randomly generated binary $H$, you first need to make sure that your $H$ is actually invertible. 

**<span style="color: red">How can you use Gaussian elimination to check that the square matrix $H$ is invertible? What must be true about the rows of $H$ for it to be invertible? What about the columns?</span>**

`YOUR ANSWER HERE`

**Note on invertibility**

Luckily, randomly generated binary matrices are *usually* invertible. However, the function we provided still double checks that the generated $H$ is indeed invertible (using an alternative method to Gaussian elimination), and re-generates the matrix if it's not.

# <a id = 'simulatorIntro'><span style = "color: blue">Task 2: Imaging Simulator</span></a>

Let's pause for a minute before we start capturing images with the projector. Recall from Imaging 2 that the projector setup is usually placed inside a cardboard box to prevent light from the outside world disturbing our sensor. Even when the projector is turned off, there might be a significant amount of light inside the box. The sensor and related circuits generate noise in our measurements due to thermal physics and other environmental factors. The refresh rate of the projector also contributes to noise. Thus, non-idealities like noise will inevitably be present in our setup. This is a limitation of the real-world setup that greatly affects our ability to reconstruct the image using the light sensor data. That's why it's important to build a simulator that **accurately models** what happens when we try to capture an image, including non-idealities that we can potentially compensate for.

Our virtual simulated projector will artificially generate noise to affect sensor results in a way that mirrors this real-world phenomenon.

## <a id = 'idealSensor'><span style = "color: blue">Task 2a: Constructing an Ideal Sensor Model</span></a>

Let's first construct a function that emulates what we would *hope* occurs when we scan an image (ideal imaging). An image (represented as the column vector $\vec{i}$) is placed in a region that can be illuminated by the projector. The projector projects a sequence of masks $H_k$ onto the image (illuminating certain pixels at a time). In our simulation, the digitized 'light sensor' output is the sum of the brightnesses detected across illuminated pixels. The $k^{th}$ entry of the sensor output vector, $s_k$, corresponds to the $k^{th}$ scan.

Recall that these operations can be represented by the previously defined mathematical model:

$$\vec{s} = H \vec{i}$$ 

**<span style="color:red"> Your first goal is to translate this ideal model into a `simulateIdealCapture` function (Fill in the TODO). Apply the function using the supplied 32x32 image of a playing card and your generated random binary matrix `H`. Display the simulated sensor reading as a 32x32 image.</span>**

*Hint: Remember to use `np.dot` to do matrix multiplication.*

The card you're trying to image should look like: <br/><br/>

<center>
<img src="images/raw_card.png" align="center" style="height:200px" />
</center>

Think about what the output sensor readings will look like. Given randomly generated masks, would you expect the output sensor readings to be remotely recognizable?

In [ ]:
# Inputs: 
#  `i2D`: 2D image you're trying to capture
#  `H`: Mask matrix
#  `matrixName`: Name of mask matrix (for image title)
#  `display`: Whether to display the sensor output as a 2D image
# Outputs:
#  `s`: Sensor reading column vector
def simulateIdealCapture(i2D, H, matrixName, display = True):
    # Number of pixels in your image = `iHeight` * `iWidth`
    iHeight = i2D.shape[0]
    iWidth = i2D.shape[1]
    iSize = iHeight * iWidth
    
    # TODO: Convert the 2D image `i2D` into a 1D column vector `i`
    i = # YOUR CODE HERE
    
    # TODO: Perform the matrix operation to emulate the ideal imaging system  --------------
    s = # YOUR CODE HERE
    
    if (display):
        
        # Reshape the simulated sensor output `s` into an appropriately 
        # sized 2D matrix `s2D` and plots it
        s2D = np.reshape(s, (iHeight, iWidth))
        plt.imshow(s2D, cmap = "gray", interpolation = "nearest")
        plt.title("Ideal Sensor Output, Using %s" % matrixName)
        plt.show()
    return s

# Load card image + display it
i2D = np.load("scripts/raw_card.npy")
plt.imshow(i2D, cmap = "gray", interpolation = "nearest")
plt.title("Raw 32x32 Image of a Playing Card")
plt.show()

# Simulate the image capture step (ideal)
s = simulateIdealCapture(i2D = i2D, H = randomH, matrixName = "Random H");

### <a id = 'idealReconstruction'><span style = "color: blue">Image Reconstruction Using the Ideal Sensor Model + Matrix Inverse</span></a>

As you can see, for *multipixel imaging*, the sensor output does not resemble the original image in any way. By applying the randomly generated mask matrix $H$, you've essentially encrypted the image data, making it unrecognizable to anyone who doesn't know the exact mask matrix $H$ you used (otherwise known as the encryption key).

If you know the key $H$, as stated before, you can reconstruct/decrypt the desired image column vector $\vec{i}$ from the sensor reading vector $\vec{s}$ by essentially *undoing* what the imaging system did to the image and applying the equation:

$$\vec{i} = H^{-1} \vec{s}$$

Again, it's important that we've selected an invertible $H$. 

**<span style="color:red">Now your job is to help write a function `idealReconstruction` (Fill in the TODO) that accepts the column vector $\vec{s}$ and mask matrix $H$ and displays the reconstructed estimate of $\vec{i}$ as a 2D image. Run the reconstruction function using the previously computed `s` and mask matrix `H` and verify that it worked as you expected.</span>**

*Hint: Use `np.linalg.inv` to invert a matrix.*

In [ ]:
# Inputs
#  `H`: Mask matrix
#  `matrixName`: Name of mask matrix (for image title)
#  `s`: Sensor reading column vector
#  `X`: Image width
#  `Y`: Image height
def idealReconstruction(H, matrixName, s, X = 32, Y = 32, realImaging = False):
    
    # TODO: Perform the matrix operations required for reconstruction --------------------
    i = # YOUR CODE HERE
    
    if realImaging:
        i = noiseMassage(i, H)
    
    # Reshape the column vector `i` to display it as a 2D image
    i2D = # YOUR CODE HERE  

    # We're going to exclude the top row and left-most column from display
    plt.imshow(i2D[1:, 1:], cmap = "gray", interpolation = "nearest")
    plt.title("Reconstructed Image, Using %s" % matrixName)
    plt.show()


# Run ideal reconstruction    
idealReconstruction(H = randomH, matrixName = "Random H", s = s)

## <a id = 'nonidealities'><span style = "color: blue">Task 2b: Handling System Non-Idealities</span></a>

The ideal reconstruction demonstrated above works great, right? Unfortunately, due to real-world non-idealities alluded to earlier, if you directly tried to image a drawing with the multipixel masks in $H$, the reconstruction would probably look terrible. A significant amount of engineering effort is focused on how to best translate theory into practice by attempting to compensate for or remove non-idealities. In the following sections, we'll look at some of the worst offenders and what we can do to improve reconstruction quality. 

### <a id = 'noiseSimulation'><span style = "color: blue">Noise *(Why So Grainy? ☹)*</span></a>

We will see noise again later in Module 3,-- if you're really interested in modelling noise and its effects, it's covered more extensively upper division classes including EE123, EE126, EE142 -- but for now it's important to realize that both the light sensor circuit and the projector add noise that shows up in the digitized sensor output. Noise is what causes photos to look grainy or fuzzy. As an example, if your single pixel imaging system from last week happened to be very noisy (and usually, the cheaper the system, the noisier it is...), imaging the card from before might've produced something like: <br/><br/>

<center>
<img src="images/noisy_card.png" align="center" style="height:200px" />
</center>

The noisier your system, the less the resultant image will look like what you expected ☹.

One way to make noise less problematic is to increase the number of pixels illuminated per scan. This increases the "signal level" (i.e. contributions from things we actually care about). At the same time, the amount of noise coming from the light sensor circuit and projector should stay mostly constant, thus improving the so-called *signal-to-noise ratio* (SNR) of our system. This is important to know when choosing `avg1sPerRow` for our random binary mask.

However, in reality, the number of pixels illuminated per scan is limited by the ambient light sensor circuit. This is because at high brightness levels, the sensor circuit becomes saturated. The difference in sensor reading for each additional illuminated pixel becomes quite small, and thus we lost the ability to differentiate the number of illuminated pixels.

Another way to make noise less problematic is to repeat each scan $k$ (with the same illumination pattern) many times and *average* the sensor outputs. The desired signal is always present, but the *random* error (noise) changes on each repeat scan. Thus, you can "average out the noise" at the expense of spending more time acquiring the image. This is actually what the Launchpad code used in lab does under the hood.

Generally speaking, we would like to build a sensing system that is as noise robust as possible, but what does that entail?

## <a id = 'eigenanalysis'><span style = "color: blue">Task 2c: Eigenanalysis $\&$ the Robustness of Inverse-Based Reconstruction</span></a>

## <span style = "color: red">THIS SECTION IS VERY IMPORTANT. PLEASE READ CAREFULLY.</span>
When noise is included, the mathematical model of our imaging system would look like:

$$ \vec{s} = H \vec{i} + \vec{\omega} +\vec{o} $$

The vector $\vec{o}$ is a vector of equal entries, which represents a constant offset from extra light from the projector while it is projecting the color black. Even though black is supposed to be an absence of light, there is still a glow present from the projector that can offset our measurement by a scalar amount. This needs to be removed, but can easily be done so by measuring and subtracting it. 

The elements ($\omega_k$) of the column vector $\vec{\omega}$ correspond to the random amounts of noise added at each measurement $s_k$. We cannot remove noise, but we can try to reduce its effects.

For example, you might expect your sensor readings $\vec{s}$ to be something like 

\begin{equation}
\vec{s_{expected}} = \begin{bmatrix}
51 \\
65 \\
42 \\
\vdots \\
32
\end{bmatrix}
\end{equation}

But you may get something like

\begin{equation}
\vec{s_{reality}} = \begin{bmatrix}
61.2 \\
76.0 \\
51.7 \\
\vdots \\
44.0
\end{bmatrix}\;.
\end{equation}

This means that what you are getting is really

\begin{equation}
    \vec{s_{reality}} = \vec{s_{expected}} \;+\; \vec{\omega} \;+\; \vec{o}
\end{equation}

\begin{equation}
    \begin{bmatrix}
        61.2 \\
        76.0 \\
        51.7 \\
        \vdots \\
        44.0
    \end{bmatrix} = 
    \begin{bmatrix}
        51 \\
        65 \\
        42 \\
        \vdots \\
        32
    \end{bmatrix}
    \;+\;
    \begin{bmatrix}
        0.2 \\
        1.0 \\
        -0.3 \\
        \vdots \\
        2.0
        \end{bmatrix}
    \;+\;
    \begin{bmatrix}
        10 \\
        10 \\
        10 \\
        \vdots \\
        10
    \end{bmatrix}
\end{equation}

where the last two vectors are $\vec{\omega}$ and $\vec{o}$

As you can see, once you measure the offset, it is very easy to just subtract from your measurements. We will take care of this for you in the experimental portion of the lab (below), so you don't need to worry about it. We will ignore it in the rest of the notebook, and assume it is subtracted.

From this point forward, our key equation will look like this:

$$ \vec{s} = H \vec{i} + \vec{\omega}$$

Now we will try to reconstruct the image $\vec{i}$ with matrix inversion $H^{-1}$:

$$ H^{-1}\vec{s} = H^{-1}H \vec{i} + H^{-1}\vec{\omega}$$

$$ H^{-1}\vec{s} = \vec{i} + H^{-1}\vec{\omega}$$

We will call $H^{-1}\vec{s} = \vec{i_{est}}$ leaving us with 
$$ \vec{i_{est}} = H^{-1} \vec{s} = \vec{i} + H^{-1} \vec{\omega} $$

Remember that we were hoping to solve for just $\vec{i}$. The additional undesired term $H^{-1} \vec{\omega}$ is what we call our reconstruction *error*, which results from linearly transforming the original noise vector $\vec{\omega}$ by $H^{-1}$. This implies that our choice of $H$ (and therefore $H^{-1}$) strongly influences how robust our overall imaging system is. 

### To build some intuition on why this is the case, recall that matrix-vector multiplication $A \vec{x} = \vec{b}$ linearly transforms $\vec{x}$ into $\vec{b}$ via scaling and rotation, as designated by $A$. Additionally, recall that the eigenvalues $\lambda_i$ and $N$ length eigenvectors $\vec{v_{\lambda_i}}$ of an $N \times N$ matrix $A$ can be found by solving for:

$$A \vec{v_{\lambda_i}} = \lambda_i \vec{v_{\lambda_i}}$$

Multiplying both sides of this equation by $A^{-1}$ and dividing by $\lambda_i$ allows us to rewrite this equation as:

$$A^{-1} \vec{v_{\lambda_i}} = \frac{1}{\lambda_i} \vec{v_{\lambda_i}} $$ 

How does this help reconstruct our image? We know that $H$, which is an $N \times N$ matrix, is invertible, and thus we know it has at most $N$ linearly independent eigenvectors. 

Our matrix $H$ also has another property, which we haven't learned about yet: it is **diagonalizable**. Diagonalizable matrices are beyond the scope of this course (but covered in EECS16B). For now, all you need to know is that a diagonalizable $N \times N$ matrix $H$ has *precisely* N linearly-independent eigenvectors.
    
So if we know $H$ has N eigenvectors, we know they span $\mathbb{R}^N$. In other words, the eigenvectors of $H$ form a basis for $\mathbb{R}^N$. Well, guess what? Our noise vector lies in $\mathbb{R}^N$. So we can write it as a linear combination of the eigenvectors like so:

$$\vec{\omega} = \alpha_1 \vec{v_1} + ... + \alpha_N \vec{v_N}$$

Now if we apply $H^{-1}$ to both sides of the equation,

$$H^{-1} \vec{\omega} = H^{-1} \alpha_1 \vec{v_1} + ... + H^{-1}\alpha_N \vec{v_N}$$

Pull out the $\alpha$ constants in front of $H^{-1}$ since scalars commute with matrices

$$H^{-1} \vec{\omega} = \alpha_1 H^{-1} \vec{v_1} + ... + \alpha_N H^{-1} \vec{v_N}$$

And we can apply the eigenvector identity shown above:

$$H^{-1} \vec{\omega} = \alpha_1 \frac{1}{\lambda_1} \vec{v_1} + ... + \alpha_N \frac{1}{\lambda_N} \vec{v_N}$$

So we can see that regardless of the scaling constants $\alpha$, if we have very large eigenvalues of $H$ then each component of $\vec{\omega}$ is attenuated, and likewise if each eigenvalue is small, the noise vector will be amplified.

### <a id = 'graphicalInterpretation'><span style = "color: blue">Graphical Interpretation</span></a>

Another way we can picture this is by showing a graphical example, thinking of how $H$ is a transformation that rotates and scales vectors. In the following image, we have our ideal sensor readings, $H\vec{i}$ and a noise vector, $\vec{\omega}$. After applying two different matrices, $H_1^{-1}$ and $H_2^{-1},$ we can see how each vector is transformed. Ideally we would want the $\vec{\omega}$ vector to be $\vec{0}$, so the recovered image is the same as the ideal reconstruction. Adding everything together to get the final result, we have $\vec{i}+H^{-1}\vec{\omega}$. Depending on the choice of $H$, the noise may end up amplified or attenuated.

<center><img src="images/2d_transform.jpg" align="center"/>
    <b>Visual representation of the effect of different matrices on the noise vector $\omega$</b>

Numerically, we can also see what happens to the noise with different matrices by applying them to a given noise vector. We will now introduce a special matrix called the <a href="https://mathworld.wolfram.com/HadamardMatrix.html" target="_blank">Hadamard matrix</a>. It has interesting properties useful in many applications. **The code below prints out the magnitude (norm) of the noise vectors after applying the inverses of the random masking matrix and the Hadamard matrix.**

In [ ]:
# Creates the random binary matrix
randomH = generateRandomBinaryMask(avg1sPerRow = 300, plot=False)
# Recall from the beginning of the lab that the H matrix refers to the randomH matrix.

# Creates the hadamardH matrix with interesting, useful properties
hadamardH = createHadamardMatrix(shape = randomH.shape, plot=False)

# Noise vector of mean 0, with standard deviation `sigma`
sigma = 7
noise = np.random.normal(0, sigma, randomH.shape[0])

# Apply hadamardH inverse to the noise vector, and then compute its norm
hadamardNorm = np.linalg.norm(np.dot(np.linalg.inv(hadamardH), noise))

# Apply randomH inverse to the noise vector, and then compute its norm
randomNorm = np.linalg.norm(np.dot(np.linalg.inv(randomH), noise))

print("Norm of the noise vector after hadamardH inverse: ", hadamardNorm)
print("Norm of the noise vector after randomH inverse: ", randomNorm)

**<span style="color:red">Which matrix amplifies the noise less?</span>**

`YOUR ANSWER HERE`

### <a id = 'revisitingIdentity'><span style = "color: blue">Revisiting the Identity Matrix</span></a>
We know that the identity matrix is invertible, but is it a good masking matrix? To answer that question, we need to know its eigenvalues.

**<span style="color:red">What are the eigenvalues of the Identity matrix? What are its eigenvalues if we scale the identity matrix by a constant? What are its eigenvectors?</span>**

`YOUR ANSWER HERE`

Scaling the eigenvalues of the identity matrix essentially boils down to "dimming" the simulated sensor readings, or making them "brighter." Think about how good your scan would be if the projector only operated on 1%, or conversely 100% of its max light intensity. It is unlikely that both would give you the same quality sensor readings. 

**Run the next block to show the ideal image, and the noise that gets added to the image. Change the constant that scales the identity from low values like 0.1 to large values like 100 to see how the noise changes with increasing or decreasing eigenvalues.**

Note that the noise is visualized in a different color scale than black and white.

In [ ]:
# Plot the thing with the image + the noise image = total image.

#########################
# CHANGE THIS VARIABLE: #
scale_factor = 1
#########################


############# Dont change anything below ###################
# Load the image
i2D = np.load("scripts/raw_card.npy")
M,N = i2D.shape

# define the mask matrix
H = scale_factor*np.eye(M*N)

# Generate a noise vector
sigma = 1.25
noise = np.random.normal(0, sigma, H.shape[0])
noise = np.reshape(noise, (M,N))

# assemble noisy measurement
s = H.dot(i2D.ravel()).reshape((M,N)) + noise
recovered_image = np.linalg.inv(H).dot(s.ravel()).reshape((M,N))

# Plot the image, noise, and image with noise
plot_image_noise_visualization(i2D, noise, s, H)


**<span style="color:red">Which scaling factor performs better: 0.01 or 1000? Why?</span>**


`YOUR ANSWER HERE`

### <a id = 'comparingScanning'><span style = "color: blue">Comparing Scanning Matrices</span></a>
Now let's take a look at the two matrices we will use to scan, hadamardH and randomH. The block of code below will show the ideal recovered image, along with the noise that gets added on top, and the total result. We will take care of generating the noise--all you have to do is tell us how much noise to add (by setting the `noise_magnitude` variable). In addition to displaying the images, the code will also print out the norm of the modified noise vector $H^{-1}\vec{\omega}$ so you can see quantitatively how different matrices impact the noise.

[comment]: <> (**<span style="color:red">First, just run the next code block so that you'll have access to `simulateCaptureWithNoise` below.</span>**)
**<span style="color:red">You will simulate the imaging system with different amounts of noise added. Run the code block below and change the noise magnitude to see how the output is affected.</span>**

In [ ]:
##### Change this #######
noise_magnitude = 50.0
#########################

############# Dont change anything below ###################
# Load the image
i2D = np.load("scripts/raw_card.npy")
M,N = i2D.shape

# define the mask matrix
randomH = generateRandomBinaryMask(avg1sPerRow = 300, plot=False)
hadamardH = createHadamardMatrix(shape = randomH.shape, plot=False)

# Generate a noise vector
sigma = noise_magnitude / np.sqrt(M*N) # noise magnitude --> std dev
noise = np.random.normal(0, sigma, randomH.shape[0])
noise = np.reshape(noise, (M,N))

# Plot the image, noise, and image with noise
plot_image_noise_visualization(i2D, noise, s, randomH, title="Reconstruction with Random $H$")
modified_noise_norm = np.linalg.norm(np.linalg.inv(randomH).dot(noise.ravel()))
print("Norm of Hinv*w = %0.4f" % (modified_noise_norm))

plot_image_noise_visualization(i2D, noise, s, hadamardH, title="Reconstruction with Hadamard $H$")
modified_noise_norm = np.linalg.norm(np.linalg.inv(hadamardH).dot(noise.ravel()))
print("Norm of Hinv*w = %0.4f" % (modified_noise_norm))

**<span style="color:red">What noise magnitudes did you have to use for each of the two matrices to make the image borderline unrecognizable?</span>**

`YOUR ANSWER HERE`

For this next section, we will examine the Hadamard matrix in a bit more detail. We will use the function `eigenanalysisComparison` that plots a histogram of the magnitudes of the eigenvalues of your $H$'s and their respective inverses (x axis = magnitude bins, y axis = number of eigenvalues within the bin's magnitude range). 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%run scripts/helpers.py
%matplotlib inline

In [ ]:
# Plot the eigenvalues of both randomH and hadamardH
eigenAnalysisComparison(H1 = randomH, matrixName1 = "Random Binary H", H2 = hadamardH, matrixName2 = "Hadamard H")

randomHInv = np.linalg.inv(randomH)
hadamardHInv = np.linalg.inv(hadamardH)

# Plot the eigenvalues of both the inverse of randomH and the inverse of hadamardH
eigenAnalysisComparison(H1 = randomHInv, matrixName1 = "Inverse of Random Binary H",\
                        H2 = hadamardHInv, matrixName2 = "Inverse of Hadamard H")

**<span style="color:red">Which of the two matrices `randomH` and `hadamardH` do you think is more noise robust and would result in a better reconstruction? Justify your answer using the eigenvalue histograms above.</span>**

`YOUR ANSWER HERE`

<a id='scanningImages'></a>
# <span style="color:blue">Task 3: Scanning Images</span>

**<span style="color:red">Before moving onto real imaging, run the code block below to generate mask matrix files.</span>**

Check that `packageMaskMatrix` generated the following files in your lab directory:
* **randomH.npy** - Saved `randomH` for restoring in case you have to close out of your Jupyter notebook. 
* **randomH_packaged.npy** - Packaged `randomH` used by `capture_image.py` **(contains extra rows)**.
* **hadamardH.npy** - Saved `hadamardH` for restoring in case you have to close out of your Jupyter notebook.
* **hadamardH_packaged.npy** - Packaged `hadamardH` used by `capture_image.py` **(contains extra rows)**.

In [ ]:
# Generate mask matrix files    
packageMaskMatrix(randomH, 'randomH')
packageMaskMatrix(hadamardH, 'hadamardH')
brightness = 90
print("Done! :)")

##  <span style="color:red">**IMPORTANT**</span>: <a id ='setup'><span style = "color: blue">Hardware Setup</span></a>

# Please follow instructions ([Cory 140](https://drive.google.com/file/d/1gw7Ju3zpzet-s4CXfgwjwEmatlzhxc8C/view?usp=sharing), [Cory 125](https://drive.google.com/file/d/1Tg2dTP-h8ziX3119otjr4TnvhwIyB2CO/view?usp=sharing)) *verbatim* to get consistent results. Having a good setup will make taking the picture much easier and cleaner. The box should in the top left of your station.

<a id='singlePixel'></a>
### <span style="color:blue">Single Pixel Sanity Check</span>

When dealing with a complicated system, it is often useful to perform a "sanity check" to make sure that a simpler subset of the system is working as expected, before adding more complexity. Let's make sure that the single pixel imager from Imaging Lab 2 works. 

**<span style="color:red">Create $H_{Single}$ for images/masks with dimensions 32x32. How many rows should it have? Note that $H_{Single}$ is the identity matrix (but it has different dimensions from $H$ in Lab 2)!</span>**

In [ ]:
# TODO: Recreate `HSingle` to scan a 32x32 image. `HSingle` is the identity matrix.
HSingle = # YOUR CODE HERE

plt.figure(figsize = (6, 6))
plt.imshow(HSingle, cmap = 'gray', interpolation = 'nearest')
np.save('HSingle.npy', HSingle)

Now that we've saved $H_{Single}$ to a file, we need an object to take a picture of. There are index cards and markers at the GSI desk; **<span style="color:red">take an index card and draw something on its blank (no lines) side. The index card should still be mostly white. Place this index card inside the box so that the projector can project onto it</span>** (see the Overview image). 

Tape is on the TA desk; do not take the whole dispenser, it should stay at the TA desk at all times.

**<span style="color:red">Double check that you've done the following before proceeding:</span>**

* Upload `AnalogReadSerial` to the Launchpad.

* Close the Energia Serial Monitor.

* Make sure that the ambient light sensor is aimed at the center of the index card! The legs should not be touching.

* Seal the imaging system inside the box to keep outside light out during scanning. If the holes for cables are too big, try to have them face a solid, unmoving object that can block out light.

### Check again that you have closed the Energia Serial Monitor. You will not be able to scan otherwise.

**<span style="color:red">You will then run the `capture_image.py` script from the following code block that projects mask patterns from $H_{Single}$ onto your image.</span>** 

When running **`capture_image.py`**, a new icon shaped like a white web will appear in the taskbar. The window depicting our masks will appear on the projector's screen. *You can confirm this by looking into the box. (Don't forget to close the box when you're done checking!)*

Before scanning, you will be prompted to specify the **COM port used by the Launchpad and the display associated with the projector.** 

**Select the COM port associated with UART1 (previously accessed in the Energia Serial Monitor and found using the Device Manager) and the 1280x720 projector screen.** Note that you want to specify the index of your choice i.e. for `2) 1280 x 720`, specify 2. **<span style="color:red">Make sure that you've selected the correct COM port!</span>** If your COM port does not show up, make sure to close Energia (since Energia can take up the port).

`capture_image.py` iterates over the rows of the $H$ matrix you made. These rows are translated, one-by-one, into real masks projected onto the screen. Light sensor readings are taken for each mask. At the start of the scan, you'll see a series of `Loc: # Data: #` printed to the output below. `Loc` corresponds to the index $k$ of the current sensor reading (and likewise current row of H). `Data` corresponds to the actual digitized value obtained from the light sensor. This "debug" information is printed consecutively for the first few/last few sensor outputs. Otherwise, this info is printed when `k % 100 = 0` (every 100 scans). 

Don't worry about messages like `Time delta between captures in s: #`. This is printed a lot because the lab computers are kind of slow. 

The whole scanning process should take roughly 3 minutes. 

*Note: On the topic of non-idealities, our scans take so long because:*
* We average sensor readings to improve the signal-to-noise ratio. Therefore we need to read more times.
* As you'll learn in Module 2, capacitors take some time to charge and discharge. A capacitor acts as a "low-pass filter." We used a 0.1$\mu$F capacitor in our sensing circuit to "smooth" the output and suppress "high-frequency" noise. In order to give the capacitor time to "settle" (i.e. ~fully charge/discharge), we need to wait longer between scans. Otherwise, the sensor reading will also include some "memory" of the previous scan result, when we really want the reading to only be about the current scan. 

**<span style="color:red">Not getting a good picture?</span>**

If you selected the incorrect COM port, a lot more "debug" lines will be dumped. You might notice `Data: -1` or `Losing data! Consider increasing your timeout!` messages. Once you see these messages, **stop** the scan, **save** your Jupyter notebook, then **close the Jupyter notebook in your Terminal (Ctrl+C twice) and re-open it**. You'll lose any saved variables (not output images), but everything you need has been saved to a file. Rerun the following code block again, but **make sure you select the right COM port**. 

In [ ]:
%run scripts/helpers.py
%matplotlib inline

%run capture_image.py --mask HSingle.npy --out sensor_readingsHSingle --width 32 --height 32

**<span style="color:red">Recreate the image from the sensor readings obtained with `HSingle`. DO NOT move on until you have an acceptable recreation. Ask for help if you need to.</span>**

*Note: Because we used 32x32 masks this time, the portion of the image we're able to "see" is smaller.*

In [ ]:
# Sensor readings
sr = np.load('sensor_readingsHSingle_100_0.npy')
HSingle = np.load('HSingle.npy')

# TODO: Create the image vector from `HSingle` and `sr`
# Hint: Because `HSingle` is a special matrix, technically you do not need to perform any matrix operations
iv = # YOUR CODE HERE

img = np.reshape(iv, (32, 32))
plt.imshow(img, cmap = 'gray', interpolation = 'nearest')

## <a id ='realImaging'><span style = "color: blue">Real Multipixel Imaging</span></a>

In the previous section, we scanned our image one pixel at a time. Now we are going to use the two matrices you examined earlier to scan. However, due to the different brightness of multi-pixel imaging, we need to change the resistance of our circuit to get better measurements.

**For Multi-Pixel Imaging**, disconnect your breadboard from power and change the 100kΩ resistor (Brown-Black-Yellow-Gold) in your circuit to a <span style = "color: red">**27kΩ (Red-Violet-Orange-Gold)**</span> resistor for better scanning quality. You can pick up the resistor from the TA Desk. Make sure to double check the rest of the system (e.g. projector) to ensure that they are not accidentally affected by this switch.

After you have switched the resistor and double-checked that the projector is running as intended, run the following blocks to scan your image with each of the two matrices.

In [ ]:
%run scripts/helpers.py
%matplotlib inline
brightness = 90
# Run scan
%run capture_image.py --mask randomH_packaged.npy --out sensor_readingsrandomH --width 32 --height 32 --brightness $brightness

sr = np.load('sensor_readingsrandomH_%s_0.npy' % str(brightness))
# Estimate the offset
oest_randomH, s_randomH = getOffsetEstimateAndS(sWithOffsetCalibration = sr)

**<span style="color:red">Let's reconstruct your image. Based off of your simulation results, is this the reconstruction quality that you expected using `H`? Think about how noisy our actual imaging system is.</span>**

In [ ]:
randomH = np.load('randomH.npy')

idealReconstruction(H = randomH, matrixName = 'randomH', s = s_randomH - oest_randomH)

**As expected, the randomly generated H matrix does not work well, if at all.**

Next, let's try to image your index card using `hadamardH`. Imaging with `hadamardH` requires some additional pre-processing to stitch sensor readings (associated with the mask that we split) back together. This has been taken care of for you in the code below. 

**<span style="color:red">Run the following code block. It will capture sensor readings using the Hadamard matrix `hadamardH`. Select the COM port associated with UART1 and the 1280x720 projector screen.</span>**

*Note: Don't worry about this error message: "`Incomplete measurement row. Is your mask matrix size correct? Padding outputs 31 times for display.`" IF it states that outputs are padded exactly 31 times. This is due to the fact that `hadamardH`'s row 0 is split.*

In [ ]:
%run scripts/helpers.py
%matplotlib inline
brightness = 90
# Run scan
%run capture_image.py --mask hadamardH_packaged.npy --out sensor_readingshadamardH --width 32 --height 32 --brightness $brightness

# PREPROCESSING -------------------------------------------------------------------

sr = np.load('sensor_readingshadamardH_%s_0.npy' % str(brightness))
# Estimate the offset
oest_hadamardH, sRow0Split = getOffsetEstimateAndS(sWithOffsetCalibration = sr)
# Combine rows 0a, 0b of `s` 
s_hadamardH = getHadamardS(sRow0Split)

<span >Don't worry if your sensor output "looks" incorrect. hadamardH is different from other matrices we've used before. We must reconstruct the image to check correctness. </span>

In [ ]:
hadamardH = np.load('hadamardH.npy')

idealReconstruction(H = hadamardH, matrixName = 'hadamardH', s = s_hadamardH - oest_hadamardH)

**<span style="color:red">Let's reconstruct your image. Based off of your simulation results, is this the reconstruction quality that you expected using `hadamardH`?</span>**

`YOUR ANSWER HERE`

**<span style="color:red">Comment on your reconstruction results when using `randomH` and `hadamardH`. In real imaging, which matrix did better? Did this match your expectations from simulation? Why? How did you expect multipixel imaging to compare to single pixel imaging from Imaging 2? What are some observed limitations of multipixel imaging?</span>**

`YOUR ANSWER HERE`

<a id='useCases'></a>
# <span style="color:blue">Task 4: Understanding Multipixel Use-Cases</span>

Based on the results you got in Task 3, you may be wondering why we'd ever use multipixel imaging at all. After all, we've significantly increased the complexity of our masks and reconstruction procedure only to produce images that don't seem to be any better (or are potentially even worse!) than before.

To highlight the advantages of multipixel imaging, let's return to our discussion of our mathematical model:

$$ \vec{s} = H \vec{i}_{est}$$

$\vec{s}$ can be interpreted as a vector of measurements: every time we project a mask onto our image and record the sensor data, it's stored as an element of $\vec{s}$. As we've seen before, solving for $\vec{i}$ yields:

$$H^{-1}\vec{s} = \vec{i}_{est}$$

If $H$ corresponds to a single-pixel setup, then 'unscrambling' (or 'inverting') $H$ will simply involve rearranging the order of elements in $\vec{s}$ (or leaving it alone if $H$ is the identity matrix). In other words, if each row of $H$ has exactly one 1 with the remaining entries equal to 0, then the same will be true of $H^{-1}$. **Because of this, with single-pixel imaging, each value in $\vec{i}_{est}$ depends on exactly one measurement!**

However, this reasoning doesn't apply to multipixel imaging. If $H$ has multiple nonzero values in each row, then we expect $H^{-1}$ to behave similarly. **As such, with multipixel imaging, each value in $\vec{i}_{est}$ can depend on several measurements.** This makes multipixel imaging more robust in certain circumstances. 

**For example, what happens if we're in a scenario where the power temporarily goes out during our scan?** In this case, we've essentially ruined a small set of our measurements. This is where we'll see a big difference between single and multipixel imaging: in single-pixel imaging, the pixels corresponding to those measurements will be irrecoverably lost, but we'll still get a decent image with multipixel imaging since each pixel's value is distributed over multiple measurements.

To illustrate this, we used the setup discussed in this lab to image this drawing:

<center><img src="images/camerashot.jpg" align="center" style="height:200px"/>

To ruin some of our measurements, we simply opened the carboard box for ten seconds during our scans. Here's the result from the single-pixel scan:

<center><img src="images/singleboxopen10s.png" align="center" style="height:200px"/>

Although you can make out most of the image, the pixels during which the box was open (the white strip) were clearly lost. Overall, this isn't a great picture. Let's see what happens when we do this with the Hadamard matrix instead:

<center><img src="images/hadamardboxopen10s.png" align="center" style="height:200px"/>

We opened the box at the same time relative to the single-pixel scan, but we've retained at least some information on most of our pixels, producing a much better image. This is because the noise is distributed over the whole image rather than just over a certain number of pixels. **As is often the case in engineering, we've encountered a tradeoff: would we rather have some pixels recovered perfectly and others not so much, or should we compromise on overall image quality so that we at least get *some* information about *every* pixel?** This is the tradeoff between single and multipixel imaging!

**<span style="color:red">In your own words, when would you choose to use multipixel imaging instead of single-pixel imaging?</span>**

`YOUR ANSWER HERE`

# Do not take apart your setup before being checked off.

**Afterwards, you may take the Launchpad and circuit kit with you. Remember to bring it back next week!**

**Please ensure you've placed the entire imaging setup box with the following items back on the station shelf:**

1. Pico Projector
2. Power Adapter
3. Long micro USB cable
4. Wooden Stand

<a id='checkoff'></a>
## Checkoff
When you are ready to get checked off, fill out the **[Checkoff Google Form](https://docs.google.com/forms/d/e/1FAIpQLSc6AEapc5xrVjHGzRgEdsHLcZBKVZ9aFR_TYMx00UNBFJ916g/viewform?usp=sf_link)**.

Your GSI or a Lab Assistant will join your breakout room when they are available and go through some checkoff questions with your group. They will go through the checkoff list in order. Please be patient!


### Post-checkoff Clean Up: (this applies to each week's lab)

1. Pack away your lab kit materials to bring home - make a note to bring them back next week!
2. Throw away any trash at your station
3. Return the wires from the back walls and any components from the TA Desk
4. SIGN OUT of the computers - DO NOT SHUT DOWN
5. Check that other machines at your lab station are powered off